In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./Landslide2.csv"
#device = "102"
#save_path = f"data/dev{device}_prepared.csv"
save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

#df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_6188\2545674397.py:13: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
0,2025-06-27 18:59:34.950000,101.0,41.500000,0.0,28.444725,79.391396,12.150635,8.638015,99.898972
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
2,2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
4,2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
...,...,...,...,...,...,...,...,...,...
308536,2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972
308537,2025-06-29 20:25:48.980000,102.0,25.600000,0.0,29.908064,77.426796,3.856516,8.637931,99.898941
308538,2025-06-29 20:26:01.100000,102.0,25.600000,0.0,29.349968,80.005569,2.614293,8.637931,99.898949
308539,2025-06-29 20:26:02.150000,101.0,27.900000,0.0,29.050890,81.302589,8.617684,8.638011,99.898972


In [6]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,301191,301191.000000,301191.000000,301191.000000,301191.000000,301191.000000,301191.000000,301191.000000,301191.000000
mean,2025-06-03 16:02:50.819243776,101.503282,48.952394,0.205600,29.871124,75.252266,8.128838,8.637978,99.898949
min,2025-05-02 11:42:00.384119,101.000000,21.100000,0.000000,24.238958,43.418247,0.000000,8.637330,99.166664
25%,2025-05-29 07:21:07.383100928,101.000000,44.200001,0.000000,27.368584,69.429153,3.009754,8.637945,99.898926
50%,2025-06-10 11:36:09.350430976,102.000000,50.400002,0.000000,28.508812,79.240715,4.383256,8.637974,99.898949
75%,2025-06-17 09:13:48.344999936,102.000000,57.799999,0.000000,32.749290,81.176704,13.366670,8.638014,99.898972
max,2025-06-29 20:26:13.090000,102.000000,66.599998,41.630600,38.984512,88.510567,32.062466,8.638361,99.900078
std,NaN,0.499990,10.397376,1.761622,3.193456,8.565651,6.716052,0.000038,0.001335


In [7]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
#df = df.sort_values(['devID', 'timestamp']).reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,lat,lng
timestamp,,,,,,,,
2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,8.637964,99.898933
2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,8.637963,99.898933
2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,8.637963,99.898933
...,...,...,...,...,...,...,...,...
2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972
2025-06-29 20:25:48.980000,102.0,25.600000,0.0,29.908064,77.426796,3.856516,8.637931,99.898941
2025-06-29 20:26:01.100000,102.0,25.600000,0.0,29.349968,80.005569,2.614293,8.637931,99.898949


In [8]:
#df = df[(np.abs(zscore(df[['soil', 'rain', 'temp', 'humi', 'geo']])) < 20).all(axis=1)]
#df

In [9]:
'''
# ฟังก์ชันเติม NaN หลัง rolling
def fill_na(df_group):
    return df_group.fillna(method='bfill').fillna(method='ffill')

# กำหนด windows
rain_windows = {
    'rain_1h_sum': '1H',
    'rain_3h_sum': '3H',
    'rain_6h_sum': '6H',
    'rain_12h_sum': '12H',
    'rain_1d_sum': '1D',
    'rain_7d_sum': '7D',
}
soil_mean_windows = {
    'soil_1h_mean': '1H',
    'soil_3h_mean': '3H',
    'soil_6h_mean': '6H',
    'soil_12h_mean': '12H',
    'soil_1d_mean': '1D',
}
geo_mean_windows = {
    'geo_1h_mean': '1H',
    'geo_3h_mean': '3H',
    'geo_6h_mean': '6H',
    'geo_12h_mean': '12H',
    'geo_1d_mean': '1D',
}

# ต้อง reset_index เพื่อให้ devID เป็น column สำหรับ groupby
df_reset = df.reset_index()

# สร้าง group ตาม devID สำหรับ rolling
grouped = df_reset.groupby('devID')

# Rolling sums/means
for col_name, window in rain_windows.items():
    df_reset[col_name] = grouped['rain'].rolling(window, on='timestamp').sum().reset_index(level=0, drop=True).values
for col_name, window in soil_mean_windows.items():
    df_reset[col_name] = grouped['soil'].rolling(window, on='timestamp').mean().reset_index(level=0, drop=True).values
for col_name, window in geo_mean_windows.items():
    df_reset[col_name] = grouped['geo'].rolling(window, on='timestamp').mean().reset_index(level=0, drop=True).values

# เติม NaN ที่เกิดจาก rolling
df_reset = grouped.apply(fill_na)
df_reset.index = df_reset.index.droplevel(0)

df = df_reset  # อัปเดต df ให้เป็น DataFrame ที่เติม rolling แล้ว

df
df_reset.index = df_reset.index.droplevel(0)

df = df_reset  # อัปเดต df ให้เป็น DataFrame ที่เติม rolling แล้ว
'''

"\n# ฟังก์ชันเติม NaN หลัง rolling\ndef fill_na(df_group):\n    return df_group.fillna(method='bfill').fillna(method='ffill')\n\n# กำหนด windows\nrain_windows = {\n    'rain_1h_sum': '1H',\n    'rain_3h_sum': '3H',\n    'rain_6h_sum': '6H',\n    'rain_12h_sum': '12H',\n    'rain_1d_sum': '1D',\n    'rain_7d_sum': '7D',\n}\nsoil_mean_windows = {\n    'soil_1h_mean': '1H',\n    'soil_3h_mean': '3H',\n    'soil_6h_mean': '6H',\n    'soil_12h_mean': '12H',\n    'soil_1d_mean': '1D',\n}\ngeo_mean_windows = {\n    'geo_1h_mean': '1H',\n    'geo_3h_mean': '3H',\n    'geo_6h_mean': '6H',\n    'geo_12h_mean': '12H',\n    'geo_1d_mean': '1D',\n}\n\n# ต้อง reset_index เพื่อให้ devID เป็น column สำหรับ groupby\ndf_reset = df.reset_index()\n\n# สร้าง group ตาม devID สำหรับ rolling\ngrouped = df_reset.groupby('devID')\n\n# Rolling sums/means\nfor col_name, window in rain_windows.items():\n    df_reset[col_name] = grouped['rain'].rolling(window, on='timestamp').sum().reset_index(level=0, drop=True).v

In [10]:
"""
# ความต่างและ STD
df['soil_diff'] = grouped['soil'].diff().reset_index(level=0, drop=True)
df['geo_diff'] = grouped['geo'].diff().reset_index(level=0, drop=True)
df['soil_1h_std'] = grouped['soil'].rolling('1H').std().reset_index(level=0, drop=True)
df['geo_1h_std'] = grouped['geo'].rolling('1H').std().reset_index(level=0, drop=True)

# เติม NaN ที่เกิดจาก rolling
df = grouped.apply(fill_na)
df.index = df.index.droplevel(0)
"""
df = df.reset_index()  # เอา timestamp กลับมาเป็น column


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301191 entries, 0 to 301190
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  301191 non-null  datetime64[ns]
 1   devID      301191 non-null  float64       
 2   soil       301191 non-null  float64       
 3   rain       301191 non-null  float64       
 4   temp       301191 non-null  float64       
 5   humi       301191 non-null  float64       
 6   geo        301191 non-null  float64       
 7   lat        301191 non-null  float64       
 8   lng        301191 non-null  float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 20.7 MB


In [12]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [17]:

df.to_csv(f"{save_path}", index=False)

print("✅ Save")

✅ Save
